In [4]:
# Instala pyreadstat para poder leer archivos .sav d
%pip install pyreadstat pandas

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement SparkSession (from versions: none)
ERROR: No matching distribution found for SparkSession


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyspark.sql (from versions: none)
ERROR: No matching distribution found for pyspark.sql


In [1]:
import os
import glob
import pandas as pd


# ⚠️ REEMPLACE ESTA RUTA con la ruta ABSOLUTA de su carpeta local
BASE_DIR = "/Users/saram/OneDrive/Documents/Data science/lab8/data" 

# Nombre del archivo maestro que crearemos
OUTPUT_FILE = os.path.join(BASE_DIR, "BASE_MAESTRA.xlsx")

# 1. Busca todos los archivos .xlsx
all_excel_files = glob.glob(os.path.join(BASE_DIR, "*.xlsx"))

if not all_excel_files:
    print("❌ ERROR: No se encontraron archivos .xlsx en la ruta especificada.")
else:
    print(f"Archivos encontrados: {len(all_excel_files)}")
    
    # 2. Lee y une todos los DataFrames
    master_list = []
    
    # Agrega una columna para rastrear el origen (opcional, pero útil)
    for f in all_excel_files:
        df = pd.read_excel(f)
        df['ORIGEN_ARCHIVO'] = os.path.basename(f)
        master_list.append(df)

    # Une todos los DataFrames en uno solo
    master_df = pd.concat(master_list, ignore_index=True)
    
    # 3. Guarda el archivo maestro en la misma carpeta
    master_df.to_excel(OUTPUT_FILE, index=False)
    
    print("\n✅ UNIÓN COMPLETA.")
    print(f"El archivo maestro se creó en: {OUTPUT_FILE}")
    print(f"Registros totales en el maestro: {len(master_df)}")

Archivos encontrados: 27

✅ UNIÓN COMPLETA.
El archivo maestro se creó en: /Users/saram/OneDrive/Documents/Data science/lab8/data\BASE_MAESTRA.xlsx
Registros totales en el maestro: 228968


In [2]:
import os
from pyspark.sql import SparkSession
from pathlib import Path
from pyspark.sql.functions import col, sum as spark_sum, count

# =======================================================================
# ⚠️ 1. CONFIGURACIÓN LOCAL Y RUTA
# =======================================================================
os.environ["HADOOP_HOME"] = "C:/hadoop" 

# 1. Iniciar la sesión de Spark con la configuración para leer Excel
spark = SparkSession.builder \
    .appName("PNC_Lab_Maestro") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

# 2. REEMPLACE ESTA RUTA con la ruta ABSOLUTA de su carpeta local (LA MISMA QUE ANTES)
BASE_DIR = "/Users/saram/OneDrive/Documents/Data science/lab8/data" 
# 3. DEFINICIÓN MANUAL DE LA RUTA MAESTRA (Fijando el formato con pathlib)
# Esto garantiza que la ruta sea interpretada correctamente por la JVM de Spark.
# Construimos la ruta y la convertimos en un URI de archivo para máxima compatibilidad.
MASTER_PATH = Path(BASE_DIR) / "BASE_MAESTRA.xlsx"
MASTER_FILE_URI = MASTER_PATH.as_uri()

# 4. Iniciar la sesión de Spark (con el conector Excel)
spark = SparkSession.builder \
    .appName("PNC_Lab_Maestro") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

# =======================================================================
# 2. CARGA Y DIVISIÓN DEL ARCHIVO MAESTRO
# =======================================================================

print("\n--- INICIO DE CARGA Y FILTRADO ---")

# Cargar el archivo maestro único
try:
    master_df = spark.read.format("com.crealytics.spark.excel") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(MASTER_FILE)
    
    # Estandariza a MAYÚSCULAS para evitar errores de columna
    for c in master_df.columns:
        if c.upper() != c:
            master_df = master_df.withColumnRenamed(c, c.upper())
            
    # Muestra todas las columnas del archivo maestro (ayuda a la clasificación)
    print("\nColumnas del archivo maestro (use estos nombres para el filtrado):")
    print(master_df.columns)

except Exception as e:
    print(f"❌ ERROR: No se pudo cargar el archivo maestro. Asegúrese de que existe en {MASTER_FILE}")
    spark.stop()
    exit() # Detiene la ejecución si falla la carga

# --- DEFINICIÓN DE COLUMNAS CLAVE POR BASE (AJUSTE ESTOS NOMBRES SI ES NECESARIO) ---

# Columna única que solo existe en la base de Vehículos (ej. una ID que no está en las otras)
COLUMNAS_VEHICULOS = ['MARCA', 'COLOR', 'TIPO_VEHICULO', 'SEXO_CONDUCTOR'] 

# Columna única que solo existe en la base de Víctimas
COLUMNAS_VICTIMAS = ['CONDICION', 'GRUPO_EDAD', 'SEXO_VICTIMA'] 

# Columna clave que debe estar en todas las bases (se usa para uniones)
COLUMNAS_JOIN = ['AÑO', 'MES', 'DIA_SEMANA', 'HORA', 'DEPARTAMENTO', 'TIPO_ACCIDENTE'] 

# --- FILTRADO PARA CREAR LOS 3 DATAFRAMES ---

# 1. HECHOS (Contiene columnas de JOIN y NINGUNA de las columnas ÚNICAS de Vehículos o Víctimas)
hechos_cols = [c for c in master_df.columns if c not in COLUMNAS_VEHICULOS and c not in COLUMNAS_VICTIMAS]
hechos_df = master_df.select(*hechos_cols).distinct() 

# 2. VEHÍCULOS (Contiene columnas de JOIN y las columnas ÚNICAS de Vehículos)
vehiculos_cols = COLUMNAS_JOIN + [c for c in master_df.columns if c in COLUMNAS_VEHICULOS]
vehiculos_df = master_df.select(*vehiculos_cols).distinct()

# 3. VÍCTIMAS (Contiene columnas de JOIN y las columnas ÚNICAS de Víctimas)
victimas_cols = COLUMNAS_JOIN + [c for c in master_df.columns if c in COLUMNAS_VICTIMAS]
victimas_df = master_df.select(*victimas_cols).distinct()


print("\n--- RESUMEN DE CARGA FINAL ---")
print(f"✅ HECHOS_DF cargado. Registros: {hechos_df.count()}")
print(f"✅ VEHICULOS_DF cargado. Registros: {vehiculos_df.count()}")
print(f"✅ VICTIMAS_DF cargado. Registros: {victimas_df.count()}")


# --------------------------------------------------------------------------
# SECCIÓN: ANÁLISIS EXPLORATORIO (EJERCICIOS 1-7)
# --------------------------------------------------------------------------

# -----------------------------------------------------
# EJERCICIO 1: Mostrar el esquema de los 3 DataFrames.
# -----------------------------------------------------
print("\n" + "="*50)
print("--- EJERCICIO 1: ESQUEMAS ---")
print("="*50)
hechos_df.printSchema()
vehiculos_df.printSchema()
victimas_df.printSchema()

# -----------------------------------------------------
# EJERCICIO 2: Mostrar la cantidad total de registros de cada DataFrame.
# -----------------------------------------------------
print("\n" + "="*50)
print("--- EJERCICIO 2: CONTEO DE REGISTROS ---")
print("="*50)
print(f"Hechos: {hechos_df.count()} registros")
print(f"Vehículos: {vehiculos_df.count()} registros")
print(f"Víctimas: {victimas_df.count()} registros")

# -----------------------------------------------------
# EJERCICIO 3: Mostrar la cantidad de nulos por columna para cada DataFrame.
# -----------------------------------------------------
def count_nulls(df, df_name):
    """Calcula el conteo de nulos por columna."""
    print(f"\n--- NULOS en {df_name} ---")
    null_counts_df = df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c) 
        for c in df.columns
    ])
    
    null_counts = null_counts_df.collect()[0].asDict()
    has_nulls = False
    for k, v in null_counts.items():
        if v > 0:
            print(f"  {k}: {v} nulos")
            has_nulls = True
    
    if not has_nulls:
        print("  (No hay nulos en las columnas seleccionadas.)")

print("\n" + "="*50)
print("--- EJERCICIO 3: CONTEO DE NULOS POR COLUMNA ---")
print("="*50)
count_nulls(hechos_df, "Hechos")
count_nulls(vehiculos_df, "Vehículos")
count_nulls(victimas_df, "Víctimas")


# -----------------------------------------------------
# EJERCICIO 4: Mostrar un describe (resumen estadístico) de los 3 DataFrames.
# -----------------------------------------------------
print("\n" + "="*50)
print("--- EJERCICIO 4: RESUMEN ESTADÍSTICO (DESCRIBE) ---")
print("="*50)
print("\nHECHOS:")
hechos_df.describe().show(truncate=False)
print("\nVEHÍCULOS:")
vehiculos_df.describe().show(truncate=False)
print("\nVÍCTIMAS:")
victimas_df.describe().show(truncate=False)

# -----------------------------------------------------
# EJERCICIO 5: Cantidad de tipos de accidentes únicos en Hechos.
# -----------------------------------------------------
tipos_unicos = hechos_df.select('TIPO_ACCIDENTE').distinct().count()
print("\n" + "="*50)
print(f"--- EJERCICIO 5: TIPOS DE ACCIDENTES ÚNICOS ---")
print("="*50)
print(f"Cantidad de Tipos de Accidente Únicos: {tipos_unicos}")

# -----------------------------------------------------
# EJERCICIO 6: Conteo de accidentes por día de la semana.
# -----------------------------------------------------
accidentes_por_dia = hechos_df.groupBy('DIA_SEMANA').agg(
    count('*').alias('TOTAL_ACCIDENTES')
).orderBy(col('TOTAL_ACCIDENTES').desc())

print("\n" + "="*50)
print("--- EJERCICIO 6: CONTEO POR DÍA DE LA SEMANA ---")
print("="*50)
accidentes_por_dia.show(truncate=False)

# -----------------------------------------------------
# EJERCICIO 7: Accidentes por mes del año.
# -----------------------------------------------------
accidentes_por_mes = hechos_df.groupBy('MES').agg(
    count('*').alias('TOTAL_ACCIDENTES')
).orderBy('MES')

print("\n" + "="*50)
print("--- EJERCICIO 7: ACCIDENTES POR MES ---")
print("="*50)
accidentes_por_mes.show(truncate=False)

# Detener la sesión de Spark
spark.stop()

ValueError: relative path can't be expressed as a file URI